In [1]:
import os  
from imp import reload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
from scipy import stats
%matplotlib inline

导入收益率和因子数据

In [2]:
stockRetDF= pd.read_excel('QandReturns.xlsx', sheet_name=u'收益率')
Qdf=pd.read_excel('./smartMoney/QandReturns.xlsx', sheet_name='Q')
#Qdf=pd.read_excel('QandReturns.xlsx', sheet_name='newQ')

In [3]:
stockRetDF=stockRetDF/100
Qdf[(Qdf==-2)|(Qdf==-1 )]=np.nan
date=stockRetDF.index;code=stockRetDF.columns
stockRet=stockRetDF.as_matrix();Q=Qdf.as_matrix()

In [4]:
groupNum = 5
rankIC = []
for i in range(Q.shape[0]-1):
    X=Q[i,]    
    Y=stockRet[i+1,]
    nanIdx=np.union1d(np.where(np.isnan(Y)==True)[0],np.where(np.isnan(X)==True)[0])
    rankIC.append(stats.spearmanr(np.delete(X,nanIdx),np.delete(Y,nanIdx))[0])

In [5]:
factorOriginal=Q
import singleFactor_RegValidity

In [6]:
[factorTvalueQn,factorRegValidityQn]=singleFactor_RegValidity.fun(stockRet,factorOriginal)

E:\MultiFactor\mfm\factor_Standardized.py:18: RuntimeWarning: invalid value encountered in greater
  thisSlice[thisSlice > upperBound[i]] = upperBound[i]
E:\MultiFactor\mfm\factor_Standardized.py:19: RuntimeWarning: invalid value encountered in less
  thisSlice[thisSlice < lowerBound[i]] = lowerBound[i]
E:\MultiFactor\mfm\singleFactor_RegValidity.py:19: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  regCoef[i]=np.linalg.lstsq(X,Y)[0][0]


In [ ]:
groupPeriod=1;regressPeriod=1
dateNumber=range(stockRet.shape[0])
import singleFactor_GroupBacktest;

In [ ]:
idx2013=np.where(date.year==2013);idx2014=np.where(date.year==2014)
idx2015=np.where(date.year==2015);idx2016=np.where(date.year==2016)
idx2017=np.where(date.year==2017)

In [ ]:
mvDF=pd.read_excel('QandReturns.xlsx', sheetname=u'市值')
industryDF=pd.read_excel('QandReturns.xlsx', sheetname=u'行业')
volatilityDF=pd.read_excel('volatility.xlsx', sheetname='Sheet1')/100
mvDF[mvDF==0]=np.nan
volatilityDF[volatilityDF==0]=np.nan
mv=mvDF.as_matrix()
volatility=volatilityDF.as_matrix()

In [ ]:
def group(factorX,factorY,groupNum):#Q是factorX,mv或动量是factorY    
    factorGroup=np.zeros([factorY.shape[0],groupNum])
    for i in range(factorY.shape[0]):
        factorSort=np.sort(factorX[i,])#np.sort 默认升序排列，nan在最后
        factorSortIdx=np.argsort(factorX[i,])                
        firstNanIdx=np.where(np.isnan(factorSort)==True)[0][0]
        factorSortIdx=factorSortIdx[:firstNanIdx]
        inGroupNum=int(round(len(factorSortIdx)/groupNum))#每组股票数
        for j in range(groupNum-1):
            factorGroup[i,j]=np.nanmean(factorY[i,factorSortIdx[inGroupNum*j:inGroupNum*(j+1)]])
        factorGroup[i,groupNum-1]=np.nanmean(factorY[i,factorSortIdx[inGroupNum*(groupNum-1):]])    
#    for j in range(groupNum):
#        pylab.plot_date(date,mvGroup[:,j],linestyle='-')
#    label = [u'第1组',u'第2组',u'第3组',u'第4组',u'第5组']
#    plt.legend(label)   
    return factorGroup

In [ ]:
mvGroup=group(Q,mv,groupNum)

In [ ]:
[backTestResult,nav]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet),np.squeeze(factorOriginal),date,groupPeriod=groupPeriod,regressPeriod=regressPeriod)

In [ ]:
pd.DataFrame(backTestResult).to_excel('btQnew.xlsx')
pd.DataFrame(nav).to_excel('navQnew.xlsx')

In [ ]:
[backTestResult2013,nav13]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet[idx2013,]),np.squeeze(factorOriginal[idx2013,]),date[idx2013],groupPeriod=groupPeriod,regressPeriod=regressPeriod)
[backTestResult2014,nav14]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet[idx2014,]),np.squeeze(factorOriginal[idx2014,]),date[idx2014],groupPeriod=groupPeriod,regressPeriod=regressPeriod)
[backTestResult2015,nav15]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet[idx2015,]),np.squeeze(factorOriginal[idx2015,]),date[idx2015],groupPeriod=groupPeriod,regressPeriod=regressPeriod)
[backTestResult2016,nav16]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet[idx2016,]),np.squeeze(factorOriginal[idx2016,]),date[idx2016],groupPeriod=groupPeriod,regressPeriod=regressPeriod)
[backTestResult2017,nav17]=singleFactor_GroupBacktest.fun(np.squeeze(stockRet[idx2017,]),np.squeeze(factorOriginal[idx2017,]),date[idx2017],groupPeriod=groupPeriod,regressPeriod=regressPeriod)

In [ ]:
mvDF=pd.read_excel('QandReturns.xlsx', sheet_name=u'市值')
industryDF=pd.read_excel('QandReturns.xlsx', sheet_name=u'行业')
#volatilityDF=pd.read_excel('volatility.xlsx', sheetname='Sheet1')/100
mvDF[mvDF==0]=np.nan
#volatilityDF[volatilityDF==0]=np.nan
mv=mvDF.as_matrix()
#volatility=volatilityDF.as_matrix()

In [ ]:
Q_mv_cor=np.zeros(len(date))
for i in range(len(date)):    
    nanIdx=np.union1d(np.where(np.isnan(Q[i,])==True)[0],np.where(np.isnan(mv[i,])==True)[0])    
    Q_mv_cor[i]=np.corrcoef(np.delete(Q[i,],nanIdx),np.delete(mv[i,],nanIdx))[0,1]

In [ ]:
Q_mv_cor.shape

In [ ]:
industryDummyDict={}
industryNum=27
for iIndustryDummyDict in range(1,industryNum):
    industryDummyDict[iIndustryDummyDict]=np.zeros(len(code))
    for iCode in range(len(code)):
        if industryDF[u'行业编号'][iCode]==iIndustryDummyDict:
            industryDummyDict[iIndustryDummyDict][iCode]=1

In [ ]:
eps=np.zeros(Q.shape)
R2=np.zeros(Q.shape[0])
for i in range(Q.shape[0]):
    Y=Q[i,:]
    X=np.c_[np.log(mv[i,:]),stockRet[i-1,:]]    
    for iIndustryDummyDict in range(1,industryNum):
        X=np.c_[X,industryDummyDict[iIndustryDummyDict]]    
    nanIdx_Y=np.where(np.isnan(Y)==True)
    nanIdx_X=np.where((np.isnan(X[:,0])==True)|(np.isnan(X[:,1])==True))    
    nanIdx=np.union1d(nanIdx_X[0],nanIdx_Y[0])
    #把nan值去掉后再代入np.linalg.lstsq  
    YdelNan=np.delete(Y, nanIdx)
    XdelNan=np.delete(X, nanIdx, axis=0)  
    l=np.linalg.lstsq(XdelNan,YdelNan)  #一元无截距回归报错
    eps[i,:]=Y-np.dot(X,l[0])
    R2[i]=1-np.nansum(eps[i,:]**2)/sum((YdelNan-np.mean(YdelNan))**2)

In [ ]:
eps.shape

In [ ]:
factorOriginal = eps

In [ ]:
[factorTvalueTT,factorRegValidityTT]=singleFactor_RegValidity.fun(stockRet,factorOriginal)

In [ ]:
print(factorTvalue)
print(factorTvalueTT)

In [ ]:
[backTestResultTT,navTT]=singleFactor_GroupBacktest.fun(stockRet,factorOriginal,date,groupPeriod=groupPeriod,regressPeriod=regressPeriod)

In [ ]:
pd.DataFrame(backTestResultTT).to_excel('btTT.xlsx')
pd.DataFrame(navTT).to_excel('navTT.xlsx')

In [ ]:
NAVdf= pd.read_excel('rankICaverageQ.xlsx', sheetname='Sheet1')
date=NAVdf[u'日期']


In [ ]:
NAV_Q=np.array(NAVdf['NAV_Q']);NAV_wind=np.array(NAVdf['NAV_wind'])
freq=12
annualizedRetQ=(NAV_Q[-1]/NAV_Q[0])**(freq/len(date))-1;
mddQ=maxDrawDown.fun(NAV_Q);
sharpeQ=np.mean(np.diff(NAV_Q)/NAV_Q[0:-1])/np.std(np.diff(NAV_Q)/NAV_Q[0:-1])*math.sqrt(freq); 
annualizedRetWind=(NAV_wind[-1]/NAV_wind[0])**(freq/len(date))-1;
mddWind=maxDrawDown.fun(NAV_wind);
sharpeWind=np.mean(np.diff(NAV_wind)/NAV_wind[0:-1])/np.std(np.diff(NAV_wind)/NAV_wind[0:-1])*math.sqrt(freq); 

In [ ]:
import maxDrawDown
import math

In [ ]:
annualizedRetQ

In [ ]:
mddQ

In [ ]:
print(factorTvalue)
print(factorTvalueQn)
print(factorTvalueTT)